In [1]:
from skyfield.api import load, N, W, wgs84, utc, Angle
from datetime import datetime
ts = load.timescale()
#t = ts.from_datetime(datetime.strptime("2024-03-17 11:42 UTC", "%Y-%m-%d %H:%M %Z").replace(tzinfo=utc))
de421 = load("de421.bsp")
earth, sun, moon = de421["earth"], de421["sun"], de421["moon"]
caledonia_wsg = wgs84.latlon(40.6318188 * N, 82.9766802 * W)
caledonia = earth + caledonia_wsg

In [2]:
def print_observe(from_obj, obj):
    pos = from_obj.at(t).observe(obj)
    alt, az, d = pos.apparent().altaz()
    ra, dec, d = pos.radec()
    print(f"{obj.target_name.split()[-1]}\nalt: {alt.degrees:.2f}\t az: {az.degrees:.1f}\n ra: {ra.hours:.1f}h\tdec: {dec.degrees:.1f}")

In [3]:
t = ts.now()
lst = caledonia_wsg.lst_hours_at(t)
print(f"{t.tt_strftime()}\nLST: {Angle(hours=lst)} ({lst:.2f}h)")
print_observe(caledonia, sun)
print_observe(caledonia, moon)

2024-03-29 13:43:22 TT
LST: 20h 40m 07.10s (20.67h)
SUN
alt: 25.91	 az: 108.8
 ra: 0.6h	dec: 3.6
MOON
alt: -5.98	 az: 244.1
 ra: 15.5h	dec: -23.4
